In [39]:
import rasterio
from rasterio.fill import fillnodata
from affine import Affine
import pandas as pd
import numpy as np
import math
from functools import partial
import climatezones as cz 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
points = pd.read_csv('tmp/points.csv')
points = points[~points.long.isnull()]

def convert_points(file, col, points):
    raster = rasterio.open(file)
    transformer = rasterio.transform.AffineTransformer(raster.transform)
    band1 = raster.read(1)
    points['x'], points['y'] = transformer.rowcol(points.long, points.lat)
    fpoints = points[points.x <= raster.height].copy()
    fpoints[col] = band1[fpoints.x, fpoints.y]
    return fpoints, raster, band1


In [61]:
def find_val(point, depth, band):
    local = band[int(point[0])-depth:int(point[0])+depth,int(point[1])-depth:int(point[1])+depth]
    bins = np.bincount(local.flatten())
    if(len(bins) > 0):
        bins[0] = 0
        return bins.argmax()
    else:
        return 0
    
def fill_zeros(points, col, band):
    depth = 1
    while (points[col] == 0).any(): 
        print("depth: ", depth)
        print("zeros: ", (points[col] == 0).sum())
        temp_find = partial(find_val, depth=depth, band=band)
        points.loc[points[col] == 0, col] = points.loc[points[col] == 0, ['x', 'y']].apply(temp_find, axis=1)
        depth = math.ceil(depth * 1.25)
        if not (points[col] == 0).any():
            break
        

In [44]:
file = 'dbs/geospatial/globalelu/World_ELU_2015.tif'
col = 'elu'
gpoints, graster, gband = convert_points(file, col, points)
fill_zeros(gpoints, col, gband)
gpoints[["gbifid", col]].to_csv("dbs/globalelu.csv", header=True, index=False)

In [62]:
beck = 'dbs/geospatial/Beck_KG_V1/Beck_KG_V1_present_0p0083.tif'
col = 'kg'
kpoints, kraster, kband = convert_points(beck, col, points)
fill_zeros(kpoints, col, kband)
kpoints[["gbifid", col]].to_csv("dbs/beck-kg-v1.csv", header=True, index=False)

depth:  1
zeros:  479806
depth:  2
zeros:  269680
depth:  3
zeros:  121228
depth:  4
zeros:  71655
depth:  5
zeros:  42782
depth:  7
zeros:  23255
depth:  9
zeros:  10527
depth:  12
zeros:  7449
depth:  15
zeros:  6080
depth:  19
zeros:  5665
depth:  24
zeros:  5342
depth:  30
zeros:  5074
depth:  38
zeros:  4798
depth:  48
zeros:  4183
depth:  60
zeros:  3686
depth:  75
zeros:  3553
depth:  94
zeros:  3452
depth:  118
zeros:  3207
depth:  148
zeros:  3124
depth:  185
zeros:  3112
depth:  232
zeros:  2760
depth:  290
zeros:  2637
depth:  363
zeros:  2602
depth:  454
zeros:  2373
depth:  568
zeros:  2123
depth:  710
zeros:  388
depth:  888
zeros:  19
depth:  1110
zeros:  16
depth:  1388
zeros:  10
depth:  1735
zeros:  3
depth:  2169
zeros:  1


In [58]:
(fpoints[col] == 0).sum()

6458

In [ ]:
(zeros.local.apply(lambda x: ).sum()

In [ ]:
mask = raster.read_masks(1)
result = fillnodata(band, mask, max_search_distance=500)

In [ ]:
newfile = 'dbs/geospatial/globalelu/World_ELU_2015_filled.tif'

with rasterio.open(newfile, 'w', **profile) as dest:
    dest.write_band(1, result)

In [ ]:
fpoints.groupby(by="elu").count()